In [1]:
# Imports

import glob
import keras
import numpy as np
import os
import random as python_random
import tensorflow as tf


from keras import backend as K
from keras import layers
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPool2D, MaxPooling2D
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import array_to_img, img_to_array, load_img, np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder



2023-01-08 15:32:54.829587: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 15:32:55.326929: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-08 15:32:55.326964: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-08 15:32:56.672885: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
# Read data

IMAGE_WIDTH = 224    
IMAGE_HEIGHT = 224
image_size=(IMAGE_WIDTH, IMAGE_HEIGHT)

train_paper = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/paper/*.jpg')
train_cardboard = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/cardboard/*.jpg')
train_plastic = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/plastic/*.jpg')
train_metal = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/metal/*.jpg')
train_trash = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/trash/*.jpg')
train_battery = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/battery/*.jpg')
train_shoes = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/shoes/*.jpg')
train_clothes = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/clothes/*.jpg')
train_glass = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/glass/*.jpg')
train_biological = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/biological/*.jpg')

train_files = [fn for fn in train_paper]+[fn for fn in train_cardboard]+[fn for fn in train_plastic]+[fn for fn in train_metal]+[fn for fn in train_trash]+[fn for fn in train_battery]+[fn for fn in train_shoes]+[fn for fn in train_clothes]+[fn for fn in train_glass]+[fn for fn in train_biological]
print(len(train_files))

train_imgs = [img_to_array(load_img(img, target_size=image_size)) for img in train_files]
train_imgs = np.array(train_imgs)
print(train_imgs.shape)

train_labels = [0 for i in range(len(train_paper))]+[1 for i in range(len(train_cardboard))]+[2 for i in range(len(train_plastic))]+[3 for i in range(len(train_metal))]+[4 for i in range(len(train_trash))]+[5 for i in range(len(train_battery))]+[6 for i in range(len(train_shoes))]+[7 for i in range(len(train_clothes))]+[8 for i in range(len(train_glass))]+[9 for i in range(len(train_biological))]
    
val_paper = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/paper/*.jpg')
val_cardboard = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/cardboard/*.jpg')
val_plastic = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/plastic/*.jpg')
val_metal = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/metal/*.jpg')
val_trash = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/trash/*.jpg')
val_battery = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/battery/*.jpg')
val_shoes = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/shoes/*.jpg')
val_clothes = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/clothes/*.jpg')
val_glass = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/glass/*.jpg')
val_biological = glob.glob('/home/manu/Documents/TP_IC/archive/garbage_classification/biological/*.jpg')

val_files = [fn for fn in val_paper]+[fn for fn in val_cardboard]+[fn for fn in val_plastic]+[fn for fn in val_metal]+[fn for fn in val_trash]+[fn for fn in val_battery]+[fn for fn in val_shoes]+[fn for fn in val_clothes]+[fn for fn in val_glass]+[fn for fn in val_biological]
val_imgs = [img_to_array(load_img(img, target_size=image_size)) for img in val_files]
val_imgs = np.array(val_imgs)
print(val_imgs.shape)
val_labels= [0 for i in range(len(val_paper))]+[1 for i in range(len(val_cardboard))]+[2 for i in range(len(val_plastic))]+[3 for i in range(len(val_metal))]+[4 for i in range(len(val_trash))]+[5 for i in range(len(val_battery))]+[6 for i in range(len(val_shoes))]+[7 for i in range(len(val_clothes))]+[8 for i in range(len(val_glass))]+[9 for i in range(len(val_biological))]

15515


In [ ]:
num_classes = 10
epochs = 10
IMAGE_CHANNELS = 3
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# encode text category labels
train_labels_array = np.array(train_labels)
le = LabelEncoder()
train_integer_encoded = le.fit_transform(train_labels_array)
ohe = OneHotEncoder(sparse=False)
train_integer_encoded = train_integer_encoded.reshape(len(train_integer_encoded), 1)
train_labels_ohe = ohe.fit_transform(train_integer_encoded)

validation_labels_array = np.array(val_labels)
validation_integer_encoded = le.fit_transform(validation_labels_array)
ohe = OneHotEncoder(sparse=False)
validation_integer_encoded = validation_integer_encoded.reshape(len(validation_integer_encoded), 1)
validation_labels_ohe = ohe.fit_transform(validation_integer_encoded)

In [ ]:
train_imgs_scaled = train_imgs.astype('float32')
validation_imgs_scaled  = val_imgs.astype('float32')
train_imgs_scaled /= 255
validation_imgs_scaled /= 255

In [ ]:
def make_model(dense_layer_sizes, filters, kernel_size, pool_size):
    '''Creates model comprised of 2 convolutional layers followed by dense layers

    dense_layer_sizes: List of layer sizes. This list has one number for each layer
    nb_filters: Number of convolutional filters in each convolutional layer
    nb_conv: Convolutional kernel size
    nb_pool: Size of pooling area for max pooling
    '''

    model = Sequential()

    model.add(Conv2D(filters, (kernel_size, kernel_size),
                     padding='valid', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(filters, (kernel_size, kernel_size)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size))
        model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    return model


In [ ]:
dense_size_candidates = [[32], [64], [32, 32], [64, 64]]
my_classifier = KerasClassifier(make_model, batch_size=32)

In [ ]:
validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': dense_size_candidates,
                                     # nb_epoch is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [3, 6],
                                     'filters': [8],
                                     'kernel_size': [3],
                                     'pool_size': [2]},
                         scoring='neg_log_loss',
                         n_jobs=1)
validator.fit(train_imgs_scaled, train_labels_ohe)